In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import chart_studio.plotly as ply
from scipy.signal import spectrogram, firwin, filtfilt, hilbert
from scipy.stats import truncnorm
from utilities.signal_analysis_utils import get_harmonic_envelope, get_multi_harmonic_envelope, get_total_envelope

**Synthetic Modeling**

Constructs a synthetic received signal, one component at a time, to test envelope isolation method.
1. Pure co-sinusoidal wave at driving frequency
2. Slow-varying modulatory envelope
3. Out-of-phase component with its own envelope
4. Harmonics
5. White noise (modeled as i.i.d)
6. Periodic noise (and harmonics)

The envelope isolation method involves "locking-in" (multiplying) the received signal with analytic representations of the driving signal at the fundamental and higher harmonic frequencies and then performing a low-pass (boxcar) filter.

In [ ]:
def gen_envelope(duration, times, mu=5, sigma=1):
    
    lower = 0
    upper = duration
    a, b = (lower - mu) / sigma, (upper - mu) / sigma  # Convert bounds to standard normal units

    return truncnorm(a, b, loc=mu, scale=sigma).pdf(times)

In [ ]:
# generating pure co-sinusoidal driving signal and received signal
driving_frequency = 100 # in [Hz]
sample_rate = int(2e3) # in [Hz]
duration = 10 # in [s]
times = np.linspace(0,duration,duration*sample_rate)
driving_fundamental_synthetic_voltages = np.cos(2*np.pi*driving_frequency*times)
driving_2ndharmonic_synthetic_voltages = np.cos(4*np.pi*driving_frequency*times)
received_outofphase_fundamental_synthetic_voltages = np.sin(2*np.pi*driving_frequency*times)
received_outofphase_2ndharmonic_synthetic_voltages = np.sin(4*np.pi*driving_frequency*times)
white_noise = np.random.normal(0, .1, duration*sample_rate) # mean of 0, sigma of .1

In [ ]:
# pure co-sinusoidal received signal
received_synthetic_voltages = driving_fundamental_synthetic_voltages

In [ ]:
# amplitude modulated co-sinusoidal received signal
received_synthetic_voltages = gen_envelope(duration, times)*driving_fundamental_synthetic_voltages

In [ ]:
# amplitude modulated co-sinusoidal received signal with modulated out-of-phase component
received_synthetic_voltages = gen_envelope(duration, times)*driving_fundamental_synthetic_voltages \
                            + 2*gen_envelope(duration, times, mu=2, sigma=1)*received_outofphase_fundamental_synthetic_voltages

In [ ]:
# amplitude modulated co-sinusoidal received signal with small modulated out-of-phase component + 2nd harmonics
received_synthetic_voltages = gen_envelope(duration, times)*driving_fundamental_synthetic_voltages \
                            + 2*gen_envelope(duration, times, mu=2, sigma=1)*driving_2ndharmonic_synthetic_voltages \
                            + gen_envelope(duration, times)*received_outofphase_fundamental_synthetic_voltages \
                            + 2*gen_envelope(duration, times, mu=2, sigma=1)*received_outofphase_2ndharmonic_synthetic_voltages

In [ ]:
# amplitude modulated co-sinusoidal received signal with small modulated out-of-phase component + 2nd harmonics + white noise
received_synthetic_voltages = gen_envelope(duration, times)*driving_fundamental_synthetic_voltages \
                            + 2*gen_envelope(duration, times, mu=2, sigma=1)*driving_2ndharmonic_synthetic_voltages \
                            + gen_envelope(duration, times)*received_outofphase_fundamental_synthetic_voltages \
                            + 2*gen_envelope(duration, times, mu=2, sigma=1)*received_outofphase_2ndharmonic_synthetic_voltages \
                            + white_noise

In [ ]:
# constructing analytic driving signals and computing "locked-in" received signal
driving_analytic_fundamental_synthetic_voltages = hilbert(driving_fundamental_synthetic_voltages)
received_locked_fundamental_synthetic_voltages = driving_analytic_fundamental_synthetic_voltages*received_synthetic_voltages
driving_analytic_2ndharmonic_synthetic_voltages = hilbert(driving_2ndharmonic_synthetic_voltages)
received_locked_2ndharmonic_synthetic_voltages = driving_analytic_2ndharmonic_synthetic_voltages*received_synthetic_voltages

# low-pass filtering to isolate envelopes
filter_coeffs = firwin(numtaps=int(sample_rate/(.01*driving_frequency)), cutoff=1, fs=sample_rate, window='boxcar')
received_filtered_locked_fundamental_synthetic_voltages = filtfilt(filter_coeffs, 1.0, received_locked_fundamental_synthetic_voltages)
received_fundamental_inphase_envelope = 2*received_filtered_locked_fundamental_synthetic_voltages.real
received_fundamental_outofphase_envelope = 2*received_filtered_locked_fundamental_synthetic_voltages.imag
received_filtered_locked_2ndharmonic_synthetic_voltages = filtfilt(filter_coeffs, 1.0, received_locked_2ndharmonic_synthetic_voltages)
received_2ndharmonic_inphase_envelope = 2*received_filtered_locked_2ndharmonic_synthetic_voltages.real
received_2ndharmonic_outofphase_envelope = 2*received_filtered_locked_2ndharmonic_synthetic_voltages.imag

# low-pass filtering to remove noise for total envelope
filter_coeffs = firwin(numtaps=int(sample_rate/(10*driving_frequency)), cutoff=200, fs=sample_rate, window='boxcar')
received_filtered_synthetic_voltages = filtfilt(filter_coeffs, 1.0, received_synthetic_voltages)
received_total_envelope = np.abs(hilbert(received_filtered_synthetic_voltages))
#received_total_envelope = np.abs(hilbert(received_synthetic_voltages))

Try other windowing functions for the filter! Also, not sure about the exact difference between filter width and cutoff frequency. The width determines how sharply frequencies above the cutoff are attenuated?

Why do we get an increasingly large time shift for our envelopes as the filter width is increased? Get Dan to explain the (M-1)/2 delay

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x= times,
    y= received_synthetic_voltages, 
    mode='lines', 
    name='Synthetic Received Signal',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    x= times,
    y= received_fundamental_outofphase_envelope, 
    mode='lines', 
    name='Fundamental Out-of-Phase Envelope Component',
    line=dict(color='gold', width=2)
))
fig.add_trace(go.Scatter(
    x= times,
    y= received_fundamental_inphase_envelope, 
    mode='lines', 
    name='Fundamental In-Phase Envelope Component',
    line=dict(color='limegreen', width=2, dash='dash')
))
fig.add_trace(go.Scatter(
    x= times,
    y = np.sqrt(received_fundamental_outofphase_envelope**2 + received_fundamental_inphase_envelope**2),
    mode='lines', 
    name='Fundamental Envelope',
    line=dict(color='royalblue', width=2)
))
fig.add_trace(go.Scatter(
    x= times,
    y= np.sqrt(received_2ndharmonic_outofphase_envelope**2 + received_2ndharmonic_inphase_envelope**2), 
    mode='lines', 
    name='2nd Harmonic Envelope',
    line=dict(color='tomato', width=2)
))
fig.add_trace(go.Scatter(
    x= times,
    y= received_total_envelope, 
    mode='lines', 
    name='Total Envelope',
    line=dict(color='gray', width=2)
))
fig.update_layout(
    title='Synthetic Received Signal vs. Time',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (V)',
    template='plotly_dark',
    hovermode='x unified',
)
fig.show()

Spectra before and after filtering

In [ ]:
received_locked_synthetic_voltages_fft = np.fft.fftshift(np.fft.fft(received_locked_fundamental_synthetic_voltages))
received_locked_synthetic_voltages_fftfreq = np.fft.fftshift(np.fft.fftfreq(len(received_locked_fundamental_synthetic_voltages),1./sample_rate))

received_filtered_locked_synthetic_voltages_fft = np.fft.fftshift(np.fft.fft(received_filtered_locked_fundamental_synthetic_voltages))
received_filtered_locked_synthetic_voltages_fftfreq = np.fft.fftshift(np.fft.fftfreq(len(received_filtered_locked_fundamental_synthetic_voltages),1./sample_rate))

fig = go.Figure()
fig.add_trace(go.Scatter(
    x= received_locked_synthetic_voltages_fftfreq,
    y= np.abs(received_locked_synthetic_voltages_fft), 
    mode='lines', 
    name='Pre-Filtering',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    x= received_filtered_locked_synthetic_voltages_fftfreq,
    y= np.abs(received_filtered_locked_synthetic_voltages_fft), 
    mode='lines', 
    name='Post-Filtering',
    line=dict(color='royalblue', width=2, dash='dash')
))
fig.update_layout(
    title='Locked-In Received Synthetic Signal Spectrum',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Power',
    template='plotly_dark',
    hovermode='x unified',
    xaxis=dict(range=[-300, 300]),
)
fig.show()



In [ ]:
#mag_path = 'data/mag_test.csv'
mag_path = 'data/mag_comb_dynamic_engi_08-19-25.csv'

sample_rate = int(2e3) # in [Hz]
driving_frequency = 100 # in [Hz]
comb_frequencies = [i for i in range(10, 110, 10)]
mag_data = pd.read_csv(mag_path)
mag_times = mag_data['Times (s)'].to_numpy()
ch1_voltages = mag_data['Ch 1 (V)'].to_numpy()
ch2_voltages = mag_data['Ch 2 (V)'].to_numpy()
primary_voltages = mag_data['Primary (V)'].to_numpy()
# AC voltages computed by subtracting means from raw voltages
ch1_AC_voltages = ch1_voltages - np.mean(ch1_voltages)
ch2_AC_voltages = ch2_voltages - np.mean(ch2_voltages)
primary_AC_voltages = primary_voltages - np.mean(primary_voltages)


In [ ]:
ch1_fundamental_inphase_envelope, \
ch1_fundamental_outofphase_envelope, \
ch1_fundamental_envelope = get_harmonic_envelope(ch1_AC_voltages, sample_rate, primary_AC_voltages, driving_frequency, 1)
ch1_2ndharmonic_envelope = get_harmonic_envelope(ch1_AC_voltages, sample_rate, primary_AC_voltages, driving_frequency, 2)[2]
ch1_3dharmonic_envelope = get_harmonic_envelope(ch1_AC_voltages, sample_rate, primary_AC_voltages, driving_frequency, 3)[2]
ch1_total_comb_envelope = get_multi_harmonic_envelope(ch1_AC_voltages, sample_rate, primary_AC_voltages, comb_frequencies, 1)[2]
ch1_total_envelope = get_total_envelope(ch1_AC_voltages, sample_rate, driving_frequency)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x= mag_times, 
    y= ch1_AC_voltages, 
    mode='lines', 
    name='Channel 1 Voltage',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times,
    y= ch1_fundamental_envelope,   
    mode='lines', 
    name='Fundamental Envelope',
    line=dict(color='royalblue', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times,
    y= ch1_fundamental_inphase_envelope,     
    mode='lines', 
    name='Fundamental In-Phase Envelope Component',
    line=dict(color='gold', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times,
    y= ch1_fundamental_outofphase_envelope,     
    mode='lines', 
    name='Fundamental Out-of-Phase Envelope Component',
    line=dict(color='gold', width=2, dash='dot')
))
fig.add_trace(go.Scatter(
    x= mag_times,
    y= ch1_2ndharmonic_envelope,     
    mode='lines', 
    name='2nd Harmonic Envelope',
    line=dict(color='limegreen', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times,
    y= ch1_3dharmonic_envelope,     
    mode='lines', 
    name='3d Harmonic Envelope',
    line=dict(color='tomato', width=2)
))
fig.add_trace(go.Scatter(
    x= mag_times,
    y= ch1_total_comb_envelope,     
    mode='lines', 
    name='Fundamental Total Envelope',
    line=dict(color='gray', width=2)
))
# fig.add_trace(go.Scatter(
#     x= mag_times,
#     y= ch1_total_envelope,     
#     mode='lines', 
#     name='Total Envelope',
#     line=dict(color='gray', width=2)
# ))
fig.update_layout(
    title='Ch 1 Voltage vs. Time',
    xaxis_title='Time (s)',
    yaxis_title='Voltage (V)',
    template='plotly_dark',
    hovermode='x unified',
#    xaxis=dict(range=[75, 110])
)
fig.show()

Why the heck is the magnitude off though??

Comments:
- Full envelope dominated by fundamental envelope
- Needed a much wider filter than initially suggested to properly smooth out high-frequency variations
- Do we want to know if the components are negative? What information does that provide? Or am I right to take the magnitude.

In [ ]:
ch1_locked_fundamental_AC_voltages = ch1_AC_voltages * hilbert(primary_AC_voltages)
filter_coeffs = firwin(numtaps=int(sample_rate/(.01*driving_frequency)), cutoff=1, fs=sample_rate, window='boxcar')
ch1_filtered_locked_fundamental_AC_voltages = filtfilt(filter_coeffs, 1.0, ch1_locked_fundamental_AC_voltages)

ch1_AC_voltages_fft = np.fft.fftshift(np.fft.fft(ch1_AC_voltages))
ch1_AC_voltages_fftfreq = np.fft.fftshift(np.fft.fftfreq(len(ch1_AC_voltages),1./sample_rate))

ch1_locked_fundamental_AC_voltages_fft = np.fft.fftshift(np.fft.fft(ch1_locked_fundamental_AC_voltages))
ch1_locked_fundamental_AC_voltages_fftfreq = np.fft.fftshift(np.fft.fftfreq(len(ch1_locked_fundamental_AC_voltages),1./sample_rate))

ch1_filtered_locked_fundamental_AC_voltages_fft = np.fft.fftshift(np.fft.fft(ch1_filtered_locked_fundamental_AC_voltages))
ch1_filtered_locked_fundamental_AC_voltages_fftfreq = np.fft.fftshift(np.fft.fftfreq(len(ch1_filtered_locked_fundamental_AC_voltages),1./sample_rate))

fig = go.Figure()
fig.add_trace(go.Scatter(
    x= ch1_AC_voltages_fftfreq,
    y= np.abs(ch1_AC_voltages_fft), 
    mode='lines', 
    name='Pre-Lock-In',
    line=dict(color='tomato', width=2)
))
fig.add_trace(go.Scatter(
    x= ch1_locked_fundamental_AC_voltages_fftfreq,
    y= np.abs(ch1_locked_fundamental_AC_voltages_fft), 
    mode='lines', 
    name='Post-lock-in, Pre-Filtering',
    line=dict(color='rebeccapurple', width=2)
))
fig.add_trace(go.Scatter(
    x= ch1_filtered_locked_fundamental_AC_voltages_fftfreq,
    y= np.abs(ch1_filtered_locked_fundamental_AC_voltages_fft), 
    mode='lines', 
    name='Post-lock-in, Post-Filtering',
    line=dict(color='royalblue', width=2, dash='dash')
))
fig.update_layout(
    title='Highbay Test Signal Spectrum',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Power',
    template='plotly_dark',
    hovermode='x unified',
    xaxis=dict(range=[-300, 300]),
    yaxis_type='log'
)
fig.show()

Why no low frequency component before lock-in? And why does it appear after lock-in but pre-filtering as comparable in magnitude to the 100 Hz peaks? Is it bc the peak is too sharp? Similarly, why no noise?